In [55]:
import pandas as pd
import numpy as np
import json
from difflib import get_close_matches

EDU_SYS = {
    "學士" : "學士班",
    "碩士" : "碩士班",
    "碩士專班" : "碩士在職專班",
    "博士" : "博士班"
}

FILTER = ["學制班別", "年級"]

EDU_SYS_DEPTS = {
    
}

STUDENT_HEAD = [
    "college_name",
    "dept_name",
    "education_system",
    "stu_no",
    "stu_name",
    "stu_year",
    "gender",
    "country_name",
    "id_b_remark"
]

def classify(deptName, depts):
    mostLikely = get_close_matches(deptName, depts)
    return mostLikely[0]
  

class Row:
    def __init__(self, configure):
        # print(configure)
        # self.col = col
        self.configure = configure
        self.area = {}
    
    def append(self, student):
        p



class Table5(Row):
    def __init__(self, configure, country_code):
        super().__init__(configure)
        self.country_code = country_code



    def append(self, values:pd.Series):
        try:
            country_code = self.country_code[self.country_code["國別"] == values["country_name"]].國別代碼.values[0]
        except:
            print(values)
            raise KeyError
        if not country_code in self.area:
            self.area[country_code] = {}
            self.area[country_code]["law"] = {} # 就學法
            self.area[country_code]["law"]["boy"] = []
            self.area[country_code]["law"]["girl"] = []

            self.area[country_code]["normal"] = {} # 一般管道
            self.area[country_code]["normal"]["boy"] = []
            self.area[country_code]["normal"]["girl"] = []

            self.area[country_code]["two"] = {} # 雙聯學制
            self.area[country_code]["two"]["boy"] = []
            self.area[country_code]["two"]["girl"] = []


        if values["id_b_remark"] in ("大學部指考生、研究所招生","甄試生", "學士直升博士", "直攻博士"):
            if values["gender_code"] == "1":
                self.area[country_code]["normal"]["boy"].append(values)
            else:
                self.area[country_code]["normal"]["girl"].append(values)
        elif values["id_b_remark"]  in ("獨招生", "博返碩", "中5生(僑,外生)"):
            if values["gender_code"]== "1":
                self.area[country_code]["law"]["boy"].append(values)
            else:
                self.area[country_code]["law"]["girl"].append(values)
 
        elif values["id_b_remark"] in ("雙聯學制"):
            if values["gender_code"] == "1":
                self.area[country_code]["two"]["boy"].append(values)
            else:
                self.area[country_code]["two"]["girl"].append(values)
               
        else:
            return None

    
    def output(self):
        outputData = []
        for key, item in self.area.items():
            configure = self.configure.copy()
            configure["中文國別(地區)代碼"] = key
            configure["外國學生人數_依就學辦法來臺_男"] = len(item["law"]["boy"])
            configure["外國學生人數_依就學辦法來臺_女"] = len(item["law"]["girl"])

            configure["外國學生人數_依一般升學管道就學者_男"] = len(item["normal"]["boy"])
            configure["外國學生人數_依一般升學管道就學者_女"] = len(item["normal"]["girl"])

            configure["在臺雙聯學制外國學生人數_男"] = len(item["two"]["boy"])
            configure["在臺雙聯學制外國學生人數_女"] = len(item["two"]["girl"])

            students = []

            for way in item:
                for gender in item[way]:
                    for i in range(len(item[way][gender])):
                        students.append(list(item[way][gender][i].get(STUDENT_HEAD).values))
            # for i in item["boy"]:
            #     students.append(list(i.get(STUDENT_HEAD).values))
                
            # for i in item["girl"]:
            #     students.append(list(i.get(STUDENT_HEAD).values))
            
            outputData.append([list(configure.values), students])
            
        # print(test)
        return outputData

    def outputMainTable(self):
        outputData = []
        for key, item in self.area.items():
            configure = self.configure.copy()
            configure["中文國別(地區)代碼"] = key
            configure["外國學生人數_依就學辦法來臺_男"] = len(item["law"]["boy"])
            configure["外國學生人數_依就學辦法來臺_女"] = len(item["law"]["girl"])

            configure["外國學生人數_依一般升學管道就學者_男"] = len(item["normal"]["boy"])
            configure["外國學生人數_依一般升學管道就學者_女"] = len(item["normal"]["girl"])

            configure["在臺雙聯學制外國學生人數_男"] = len(item["two"]["boy"])
            configure["在臺雙聯學制外國學生人數_女"] = len(item["two"]["girl"])

            outputData.append(configure)
        
        return outputData


def constructDataStructure(config:pd.DataFrame, country_code):
    data = {}
    for i in range(len(config.values)):
        dept = config.loc[i]["單位"]
        edu_system = config.loc[i]["學制班別"]
        grade = config.loc[i]["年級"]

        if not edu_system in data:
            data[edu_system] = {}
        
        if not dept in data[edu_system]:
            data[edu_system][dept] = {}
        
        if not grade in data[edu_system][dept]:
            data[edu_system][dept][grade] = Table5(config.loc[i], country_code)

    # print(data.keys())

    return data



def cleanData(studentData:pd.DataFrame):
    studentData.id_a_remark = studentData.id_a_remark.str.strip(' ')
    studentData.id_b_remark = studentData.id_b_remark.str.strip(' ')
    studentData = studentData[studentData.id_a_remark == '外國學生']
    studentData.education_system = studentData.education_system.str.strip(' ')
    studentData.country_name = studentData.country_name.str.strip(' ')
    studentData.dept_name = studentData.dept_name.str.replace(r' ', '')
    studentData.tmp_dept_name = studentData.dept_name.str.replace(r'\w士班', '')
    studentData.tmp_dept_name = studentData.tmp_dept_name.str.replace(r'應用數學', '')
    studentData.tmp_dept_name = studentData.tmp_dept_name.str.replace(r'碩士在職專班', '')
    studentData.tmp_dept_name = studentData.tmp_dept_name.str.replace(r' ', '')
    studentData.stu_name = studentData.stu_name.str.strip(r' ')
    studentData.college_name = studentData.college_name.str.strip(r' ')
    studentData.fillna('', inplace=True)
    return studentData


def init(config):
    global EDU_SYS_DEPTS
    EDU_SYS_DEPTS["學士"] = list(set(config.query("學制班別 == '學士班'").單位))
    EDU_SYS_DEPTS["碩士"] = list(set(config.query("學制班別 == '碩士班'").單位))
    EDU_SYS_DEPTS["博士"] = list(set(config.query("學制班別 == '博士班'").單位))
    EDU_SYS_DEPTS["碩士專班"] = list(set(config.query("學制班別 == '碩士在職專班'").單位))

**Read tables**

In [56]:
config = pd.read_excel("./空白﹍第11003期_學5.外國學生(3月、10月填報) (1).xls", dtype=str)
country_code = pd.read_excel("./國別代碼對照表 (6).xls")
studentData = pd.read_excel("./1100318 校庫學生﹍for國際處﹍外生更新1人﹍國際處更新校資組又更新國際護理博士班.xlsx",sheet_name=1, dtype=str)

**Clean data**

In [57]:
config.fillna('', inplace=True)
init(config)
dataStructure = constructDataStructure(config, country_code)
studentData["tmp_dept_name"] = np.full(studentData.shape[0], np.nan)
studentData = cleanData(studentData)

<ipython-input-55-b7b588fc27a1>:176: FutureWarning: The default value of regex will change from True to False in a future version.
  studentData.tmp_dept_name = studentData.dept_name.str.replace(r'\w士班', '')


In [58]:
studentData.tmp_dept_name

0            政治學系政治經濟學
1          外國語文學系語言學與外
2                 歷史學系
3                 歷史學系
4               電機工程學系
             ...      
1035    臨床藥學與藥物科技研究所碩士
1036           臨床醫學研究所
1037    食品安全衛生暨風險管理研究所
1038            物理治療學系
1039           公共衛生研究所
Name: tmp_dept_name, Length: 1040, dtype: object

In [59]:
dept_maps = {}
for i in range(len(studentData.values)):
    try:
        dept_name = classify(studentData.at[i, "tmp_dept_name"], EDU_SYS_DEPTS[studentData.iloc[i]["education_system"]])
    except IndexError:
        print(studentData.iloc[i]["tmp_dept_name"])
        raise IndexError
    dept_maps[studentData.at[i, "dept_name"]] = dept_name
    
with open("dept_maps.json", "w") as file:
    file.write(json.dumps(dept_maps, indent=4, ensure_ascii=False))

In [61]:
for i in range(len(studentData.values)):
    edu_sys = EDU_SYS[studentData.loc[i]["education_system"]]
    dept_name = dept_maps[studentData.loc[i]["dept_name"]]
    stu_year = studentData.loc[i]["stu_year"]
    dataStructure[edu_sys][ dept_name ][ stu_year ].append(studentData.iloc[i])

statisticData = []
tableData = []
for i in dataStructure:
    for j in dataStructure[i]:
        for k in dataStructure[i][j]:
            statisticData += dataStructure[i][j][k].output()
            tableData += dataStructure[i][j][k].outputMainTable()


newdf = pd.DataFrame(tableData)
# head = list(config.columns)
# head.append('僑居地國別')

In [62]:
newdf.to_excel("output5.xlsx")